In [ ]:
#import minsearch
import json
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
with open('documents.json', 'rt') as file_input:
    docs_raw = json.load(file_input)
docs_raw

In [ ]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
documents[7]  

In [ ]:
# index = minsearch.Index(
#     text_fields = ["question","text","section"],
#     keyword_fields = ["course"]
    
# )

# index.fit(documents)

In [ ]:
query = "the course has already started can i still enroll?"

In [ ]:
# def search(query):
#     boost = {"question": 3.0, "section": 0.5}
    
#     results = index.search(
#         query=query,
#         filter_dict={"course": "data-engineering-zoomcamp"},
#         boost_dict=boost,
#         num_results=5
#         )
#     return results
# results = search(query)

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    use only the facts from the CONTEXT when answering the QUESTION.
   
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
client = MistralClient()

In [ ]:
def llm(prompt):
    chat_response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=prompt)]
    )

    return chat_response.choices[0].message.content

In [ ]:
# def rag(query):
#     search_results = search(query)
#     prompt = build_prompt(query, search_results)
#     answer = llm(prompt)
#     return answer

In [ ]:
# #query = "the course has already started can i still enroll?"
# rag(query)

In [ ]:
# rag("do i need to know python before i enroll for the course")

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-queston"

es_client.indices.create(index=index_name, body=index_setting)


In [ ]:
documents[0]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = "do i need to know python before enrolling for the course"

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag(query)